# Templates NLP

##  1- Librairies et Packages

In [2]:
! pip install nltk

In [3]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.simplefilter(action='ignore')
import nltk
from nltk.stem import WordNetLemmatizer

### 2- Chargement de mon fichier csv

In [4]:
df = pd.read_csv("anthems.csv", encoding='utf-8')
df

,Country,Alpha-2,Alpha-3,Continent,Anthem
0,Albania,AL,ALB,Europe,"Around our flag we stand united, With one wish..."
1,Armenia,AM,ARM,Europe,"Our Fatherland, free, independent, That has fo..."
2,Austria,AT,AUT,Europe,"Land of mountains, land by the river, Land of ..."
3,Azerbaijan,AZ,AZE,Europe,"Azerbaijan, Azerbaijan! The glorious Fatherlan..."
4,Belarus,BY,BLR,Europe,"We, Belarusians, are peaceful people, Wholehea..."
...,...,...,...,...,...
185,Tunisia,TN,TUN,Africa,O defenders of the Homeland! Rally around to t...
186,Uganda,UG,UGA,Africa,"Oh Uganda! May God uphold Thee, We lay our fut..."
187,Western Sahara,EH,ESH,Africa,"O sons of the Sahara! In the battlefield, you ..."
188,Zambia,ZM,ZMB,Africa,"Stand and sing of Zambia, proud and free, Land..."


In [5]:
### Récupération de mon df
df = df[['Continent', 'Anthem']]

In [6]:
df

,Continent,Anthem
0,Europe,"Around our flag we stand united, With one wish..."
1,Europe,"Our Fatherland, free, independent, That has fo..."
2,Europe,"Land of mountains, land by the river, Land of ..."
3,Europe,"Azerbaijan, Azerbaijan! The glorious Fatherlan..."
4,Europe,"We, Belarusians, are peaceful people, Wholehea..."
...,...,...
185,Africa,O defenders of the Homeland! Rally around to t...
186,Africa,"Oh Uganda! May God uphold Thee, We lay our fut..."
187,Africa,"O sons of the Sahara! In the battlefield, you ..."
188,Africa,"Stand and sing of Zambia, proud and free, Land..."


### 3- Nettoyage des caractères spéciaux

In [7]:
### Fonction globale de nettoyage
def text_processing(texte):
    import re
    #sentence += " "
    cleaned=clean_text(texte)
    resultat = nett_caractere_spe(cleaned)
    resultat = remp_emoticon(resultat)
    resultat = remp_abreviation(resultat)
    resultat = supp_lettre_seule(resultat)
    resultat = re.sub('\n+', ' ', resultat.lower())
    resultat = re.sub(r'\d+', '', resultat)
    tokens = tokenize(resultat)
    #tokens = normalyse(cleaned)
    tokens = remove_stopwords(tokens)
    tokens = remove_Punctuation(tokens)
    tokens = lemmatize(tokens)
    return tokens
    
def clean_text(text):
    import re  #permet de traiter des donnees textuelles
    #https://docs.python.org/fr/3/library/re.html

    # lowers
    text = text.lower()
    # removes special chars
    text = re.sub(r'\W|_', ' ', text)
    # removes digits
    text = re.sub(r'[0-9]', '', text)
    # removes multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # removes space at the start or end of the string
    text = re.sub(r'^\s|\s$', '', text)

    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', ' ', text)
    
     # supprimer les liens
    text = re.sub(r'https?:\S+|http?:\S', ' link ', text)
    
    # Remplacer les mentions @XXX par ' user '
    text = re.sub(r"^@\S+|\s@\S+", ' user ', text)
    
    # Remplacer les #XXX par ' hashtags
    text = re.sub(r"^#\S+|\s#\S+", ' hashtag ', text)
    return text

def remp_emoticon(texte):  # Remplacer les emoticones par leur signification
    import re
    # mettre \ devant : ) ^
    # pas de \ devant : ; : - ] >
    resultat = re.sub(
        ';p|;P|:p|:P|xp|xP|=p|=P|:‑P|X‑P|x‑p|:‑p|:‑Þ|:‑þ|:‑b|>:P|d:|:b|:þ|:Þ',
        ' emoticon_langue ', texte)
    resultat = re.sub(':"D', 'emoticon_joyeux', resultat)
    resultat = re.sub(
        ":‑\)|:\)|:-]|:]|:->|:>|8-\)|8\)|:-}|:}|:o\)|:c\)|:\^\)|=]|=\)|:-\)\)|:'‑\)|:'\)",
        ' emoticon_joyeux ', resultat)
    resultat = re.sub(':‑D|:D|8‑D|8D|=D|=3|B\^D|c:|C:|x‑D|xD|X‑D|XD',
                      ' emoticon_rire ', resultat)
    resultat = re.sub(
        ":‑\(|:\(|:‑c|:c|:‑<|:<|:‑\[|:\[|>:\[|:{|:@|:\(|;\(|:'‑\(|:'\(|:=\(|v.v",
        ' emoticon_triste ', resultat)
    resultat = re.sub("D‑':|D:<|D:|D8|D;|D=|DX", ' emoticon_degout ', resultat)
    resultat = re.sub(
        ":‑O|:O|:‑o|:o|:-0|8‑0|>:O|=O|=o|=0|O_O|o_o|O-O|o‑o|O_o|o_O",
        ' emoticon_surprise ', resultat)
    resultat = re.sub(":-3|:3|=3|x3|X3|>:3", ' emoticon_chat ', resultat)
    resultat = re.sub(":-\|:\|:×|<3", ' emoticon_amour ', resultat)
    resultat = re.sub(";‑\)|;\)|\-\)|\\)|;‑]|;]|;\^\)|;>|:‑,|;D|;3|:‑J",
                      ' emoticon_clindoeil ', resultat)
    resultat = re.sub(":-/ |>.<|>_<|:/|:‑.|>:\|>:/|:\|=/|=\|:L|=L|:S",
                      ' emoticon_sceptique ', resultat)
    resultat = re.sub(
        "<<|>>|<.<|>.>|:$|://|://3|:‑X|:X|:‑#|:#|:‑&|:&|%‑\)|%\)",
        ' emoticon_embarrasse ', resultat)
    resultat = re.sub("8-X|8=X|x-3|x=3|X_X|x_x", ' emoticon_mort ', resultat)

    return resultat

def remp_abreviation(phrase):  # Remplacer les abréviations
    import re
    # specific
    phrase = phrase.lower()
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"cannot", "can not", phrase)
    phrase = re.sub(r"didnt", "did not", phrase)
    phrase = re.sub(r"couldnt", "could not", phrase)
    phrase = re.sub(r"doesnt", "does not", phrase)
    phrase = re.sub(r"dont", "do not", phrase)
    phrase = re.sub(r"hasnt", "has not", phrase)
    phrase = re.sub(r"hadnt", "had not", phrase)
    phrase = re.sub(r"havent", "have not", phrase)
    phrase = re.sub(r"ive", "i have", phrase)
    phrase = re.sub(r"im", "i am", phrase)
    phrase = re.sub(r"wasnt", "was not", phrase)
    phrase = re.sub(r"werent", "were not", phrase)
    phrase = re.sub(r"'cause", "because", phrase)
    phrase = re.sub(r"cos", "because", phrase)
    phrase = re.sub(r"f\*\*k", "fuck", phrase)
    phrase = re.sub(r"f\*\*king", "fucking", phrase)
    phrase = re.sub(r"idk", "i do not know", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"n\'", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r" u ", " you ", phrase)
    phrase = re.sub(r" ur ", " your ", phrase)
    phrase = re.sub(r" n ", " and ", phrase)

    return phrase

def nett_caractere_spe(texte):
    import re
    resultat = re.sub('&gt;', '>', texte)
    resultat = re.sub('&lt;', '<', resultat)
    resultat = re.sub('&quot;', '\"', resultat)
    resultat = re.sub('&amp;', '&', resultat)
    return resultat

def supp_lettre_seule(
        texte
):  # supprime les lettres seules (les abbréviations type "u", "y r")
    resultat = " ".join(
        [v for v in texte.split(" ") if ((len(v) > 1) or (not v.isalpha()))])
    return resultat

"""
Tokenisation : séparation du texte en mots
"""
def tokenize(text):
    """La tokenization, qui désigne le découpage en mots des différents documents qui constituent votre corpus"""
    from nltk.tokenize import word_tokenize

    tokens= word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    import nltk
    stopwords = nltk.corpus.stopwords.words("english") 
    return [token for token in tokens if token not in stopwords]

def remove_Punctuation(tokens):
    import string
    return [t for t in tokens if t not in string.punctuation]

''''
La radicalisation utilise la racine du mot, tandis que la lemmatisation 
utilise le contexte dans lequel le mot est utilisé. 

Le stemming consiste à réduire un mot dans sa forme « racine ».
Le but du stemming est de regrouper de nombreuses variantes d’un mot comme un seul et même mot.
'''
from nltk.stem.snowball import SnowballStemmer
import nltk

# Text Stemming
def stem(tokens):
    return [nltk.PorterStemmer().stem(token) for token in tokens]

#Text Lemmatization
def lemmatize(tokens):
    return [nltk.WordNetLemmatizer().lemmatize(token) for token in tokens]

### 4- Vectorisation

In [8]:
def text_vectorization(data,col,size): #data=input data, col=colonne text, size=nombre de mots à considerer
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf_vect = TfidfVectorizer(analyzer=text_processing,max_features=size)
    X = tfidf_vect.fit_transform(data[col])
    data_text = pd.DataFrame(X.toarray())
    data_text.columns = tfidf_vect.get_feature_names()  #names_out
    data[data_text.columns]=data_text
    data = data.drop(columns=col)
    return data

In [9]:
# all  #vectorization = regarder le nombre de fois ou le mot est sorti dans le paragraphe  et calcule sa freq
def preprocessing(data):
    #data=encodage_target_binary(data,col="Continent'")  #reequilibrage ## col : le nom de la target
    data=text_vectorization(data, "Anthem",size=15) ## Le nom de la colonne (features xi)
    return data

In [10]:
preprocessing(df)

,Continent,country,free,glory,god,homeland,land,let,li,may,nation,one,people,shall,thy,u
0,Europe,0.000000,0.000000,0.274795,0.234910,0.000000,0.199982,0.000000,0.254167,0.000000,0.243287,0.792131,0.000000,0.279377,0.000000,0.000000
1,Europe,0.000000,0.626211,0.000000,0.000000,0.000000,0.000000,0.569186,0.282468,0.000000,0.270376,0.293444,0.000000,0.000000,0.000000,0.211864
2,Europe,0.000000,0.242615,0.000000,0.000000,0.000000,0.861065,0.220521,0.000000,0.000000,0.000000,0.000000,0.208022,0.000000,0.000000,0.328333
3,Europe,0.000000,0.000000,0.000000,0.000000,0.000000,0.070772,0.000000,0.179895,0.178563,0.000000,0.093443,0.000000,0.000000,0.957842,0.067465
4,Europe,0.000000,0.000000,0.601445,0.000000,0.000000,0.218850,0.000000,0.278148,0.276088,0.000000,0.000000,0.660892,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Africa,0.000000,0.000000,0.757456,0.000000,0.223288,0.137809,0.352933,0.350298,0.000000,0.167651,0.000000,0.166465,0.192522,0.000000,0.131370
186,Africa,0.296017,0.305974,0.000000,0.255120,0.000000,0.651560,0.000000,0.276034,0.273990,0.000000,0.000000,0.000000,0.000000,0.367432,0.207039
187,Africa,0.000000,0.000000,0.000000,0.000000,0.174643,0.107786,0.000000,0.000000,0.000000,0.131127,0.711573,0.650994,0.000000,0.000000,0.102750
188,Africa,0.000000,0.627742,0.000000,0.174470,0.000000,0.222792,0.095096,0.000000,0.000000,0.180691,0.686377,0.089706,0.000000,0.000000,0.070794


In [23]:
## 
# 1) Créer une matrice des variables indépendantes et le vecteur de la variable dépendante.
# X est la matrice et Y est le vecteur
# PS: Astuce : Très souvent la dernière colonne est la colonne dépendante le y à prédire
# La matrice des variables indépendantes est aussi appeelée matrice de featuresµ
X = df.drop('Continent', axis=1)  # Supprimer la colonne "target" de la matrice X
Y = df['Continent']              # Sélectionner uniquement la colonne "target" pour Y
X

,Anthem,country,free,glory,god,homeland,land,let,li,may,nation,one,people,shall,thy,u
0,"Around our flag we stand united, With one wish...",0.000000,0.000000,0.274795,0.234910,0.000000,0.199982,0.000000,0.254167,0.000000,0.243287,0.792131,0.000000,0.279377,0.000000,0.000000
1,"Our Fatherland, free, independent, That has fo...",0.000000,0.626211,0.000000,0.000000,0.000000,0.000000,0.569186,0.282468,0.000000,0.270376,0.293444,0.000000,0.000000,0.000000,0.211864
2,"Land of mountains, land by the river, Land of ...",0.000000,0.242615,0.000000,0.000000,0.000000,0.861065,0.220521,0.000000,0.000000,0.000000,0.000000,0.208022,0.000000,0.000000,0.328333
3,"Azerbaijan, Azerbaijan! The glorious Fatherlan...",0.000000,0.000000,0.000000,0.000000,0.000000,0.070772,0.000000,0.179895,0.178563,0.000000,0.093443,0.000000,0.000000,0.957842,0.067465
4,"We, Belarusians, are peaceful people, Wholehea...",0.000000,0.000000,0.601445,0.000000,0.000000,0.218850,0.000000,0.278148,0.276088,0.000000,0.000000,0.660892,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,O defenders of the Homeland! Rally around to t...,0.000000,0.000000,0.757456,0.000000,0.223288,0.137809,0.352933,0.350298,0.000000,0.167651,0.000000,0.166465,0.192522,0.000000,0.131370
186,"Oh Uganda! May God uphold Thee, We lay our fut...",0.296017,0.305974,0.000000,0.255120,0.000000,0.651560,0.000000,0.276034,0.273990,0.000000,0.000000,0.000000,0.000000,0.367432,0.207039
187,"O sons of the Sahara! In the battlefield, you ...",0.000000,0.000000,0.000000,0.000000,0.174643,0.107786,0.000000,0.000000,0.000000,0.131127,0.711573,0.650994,0.000000,0.000000,0.102750
188,"Stand and sing of Zambia, proud and free, Land...",0.000000,0.627742,0.000000,0.174470,0.000000,0.222792,0.095096,0.000000,0.000000,0.180691,0.686377,0.089706,0.000000,0.000000,0.070794


In [19]:
Y

0      Europe
1      Europe
2      Europe
3      Europe
4      Europe
        ...  
185    Africa
186    Africa
187    Africa
188    Africa
189    Africa
Name: Continent, Length: 190, dtype: object

In [24]:
### Séparation du dataset en training_set et en test_set

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 1/3, random_state= 0)

In [25]:
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

                                                Anthem   country      free  \
129  I am ready to give life for our native hearth ...  0.344126  0.000000   
159  All hail, Liberia, hail! (All hail!) All hail,...  0.185879  0.000000   
135  O Fatherland, we shall never forget The heroes...  0.182692  0.000000   
80   Australians all let us rejoice For we are youn...  0.000000  0.179619   
97   Thou art the ruler of the minds of all people,...  0.000000  0.000000   
..                                                 ...       ...       ...   
103  Long live the King! Long live the King! His po...  0.000000  0.000000   
67   At last we reached victory In the joyous field...  0.000000  0.000000   
117  Blessed be the sacred land, Happy be the bount...  0.276373  0.000000   
47   Beloved Homeland, receive the vows That Chile ...  0.000000  0.483962   
172  Rwanda, our beautiful and dear country Adorned...  0.399194  0.206311   

        glory       god  homeland      land       let        li

# MODELES  

In [ ]:
### Regression logistique